In [3]:
import numpy as np
import cPickle as pickle
from collections import defaultdict
import sys, re
import pandas as pd
import h5py

In [19]:
train_file = "../data/train.txt"
test_file = "../data/test.txt"
valid_file = "../data/valid.txt"

gram_size = 5

def process(files, gram_size):
    eos = 1
    vocab = {'/s':eos} # don't zero index
    rev_vocab = {eos: '/s'}
    counter = eos+1
    datas = []
    targets = []
    
    for file_n in files:
                
        target = []
        data = []
        text = []
        with open(file_n, "rb") as f:
        # use collections lib to build vocab super quickly?

            for line in f:
                ints = []
                words = line.split()
                for word in words:
                    try:
                        ints.append(vocab[word])
                    except:
                        vocab[word] = counter
                        rev_vocab[counter] = word
                        counter += 1
                        ints.append(vocab[word])

                    '''
                    if word not in vocab.keys():
                        vocab[word] = counter
                        counter += 1
                    ints.append(vocab[word])
                    '''
                text.append(ints)
            print "Done vocabularizing document"
                
            for line in text:
                for i in xrange(len(line) - gram_size+1):
                    data.append(line[i:i+gram_size])
                    if i+gram_size >= len(line):
                        target.append(eos)
                    else:
                        target.append(line[i+gram_size])
        targets.append(target)

        datas.append(data)
        print "Processed", file_n  
    
    return datas, targets, vocab, rev_vocab

data, targets, vocab, rev = process([train_file, test_file, valid_file], gram_size)

train = np.array(data[0])
train_t = np.array(targets[0])
test = np.array(data[1])
test_t = np.array(targets[1])
valid = np.array(data[2])
valid_t = np.array(targets[2])

Done vocabularizing document
Processed data/train.txt
Done vocabularizing document
Processed data/test.txt
Done vocabularizing document
Processed data/valid.txt


In [8]:
#pickle.dump(vocab, open('vocab.pickle', 'wb'))

with h5py.File("language5.hdf5", "w") as f:
    dset1 = f.create_dataset("train", data=train)
    dset2 = f.create_dataset("train_t", data=train_t)
    dset3 = f.create_dataset("test", data=test)
    dset4 = f.create_dataset("test_t", data=test_t)
    dset5 = f.create_dataset("valid", data=valid)
    dset6 = f.create_dataset("valid_t", data=valid_t)

In [7]:
f = h5py.File("language_preds.hdf5")

predictions = f['predictions']

In [26]:
for i in xrange(1, 50):
    print i,
    for j in xrange(len(test[i])):
        print rev[test[i][j]],
    print rev[test_t[i]], rev[predictions[i]]

1 it was n't black monday /s /s
2 but while the new york stock /s
3 while the new york stock exchange stock
4 the new york stock exchange did exchange
5 new york stock exchange did n't composite
6 york stock exchange did n't fall n't
7 stock exchange did n't fall apart comment
8 exchange did n't fall apart friday /s
9 did n't fall apart friday as /s
10 n't fall apart friday as the 's
11 fall apart friday as the dow much
12 apart friday as the dow jones <unk>
13 friday as the dow jones industrial jones
14 as the dow jones industrial average industrial
15 the dow jones industrial average plunged average
16 dow jones industrial average plunged N fell
17 jones industrial average plunged N points N
18 industrial average plunged N points most points
19 average plunged N points most of to
20 plunged N points most of it <unk>
21 N points most of it in the
22 points most of it in the was
23 most of it in the final N
24 of it in the final hour <unk>
25 it in the final hour it <unk>
26 in the fin